In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

import random
import string
import numpy as np

import sys, os

import torch.utils.data as data

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

all_characters = string.printable
number_of_characters = len(all_characters)
# telugu_fobj = open('telugu_data.txt')
# telugu_str = telugu_fobj.read()

def character_to_label(character):
    ''' Returns the index of the character in the dictionary'''
    
    character_label = all_characters.find(character)   
    return character_label

def string_to_labels(character_string):
    ''' Returns a list of indcies for each character in the word'''
    
    return map(lambda character: character_to_label(character), character_string)

def pad_sequence(seq, max_length, pad_label=100):
    ''' Pads the sequence to have sequences of equal length'''
    
    seq += [pad_label for i in range(max_length - len(seq))]
    return seq


class LyricsGenerationDataset(data.Dataset):
    ''' Creates the data to be fed into the RNN'''
    
    def __init__(self, csv_file_path, minimum_song_count=10, artists=["Tyagaraja"], part=None):
        
        
        self.lyrics_dataframe = pd.read_csv(csv_file_path)
        self.section = part
        
        if artists:    
            self.lyrics_dataframe = self.lyrics_dataframe[self.lyrics_dataframe.artist.isin(artists)]
            self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        
            
        self.lyrics_dataframe = self.lyrics_dataframe[part].dropna()
        self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        if part == "pallavi":
            self.max_text_len = self.lyrics_dataframe.pallavi.map(lambda x: len(x)).max()
        if part == "anupallavi":
            self.max_text_len = self.lyrics_dataframe.anupallavi.map(lambda x: len(x)).max()
        if part == "f_caranam":
            self.max_text_len = self.lyrics_dataframe.f_caranam.map(lambda x: len(x)).max()
        if part == "l_caranam":
            self.max_text_len = self.lyrics_dataframe.l_caranam.map(lambda x: len(x)).max()
        if part == "text":
            self.max_text_len = self.lyrics_dataframe.text.map(lambda x: len(x)).max()
            
        # Length of biggest lyrics to be used for padding
#         self.max_text_len = int(self.lyrics_dataframe.part.str.len().max())
        print ("Part:", part)
        print ("max_text_len", self.max_text_len)
        
        # Number of songs
        whole_dataset_len = len(self.lyrics_dataframe)
        
        # Create indices array of the size - number of songs
        self.indexes = range(whole_dataset_len)
        
        # List of artists in the csv file
#         self.artists_list = list(self.lyrics_dataframe.artist.unique())
        
        # Number of artists
#         self.number_of_artists = len(self.artists_list)
    
    
    def __len__(self):
        
        return len(self.indexes)
    
    
    def __getitem__(self, index):    # obj[i] basically calls get_item function
        ''' Returns a tuple of padded input_string, output_string and the sequence length'''   
        
        # Get the text of the song... Each sequence is a text of a single song
        index = self.indexes[index]
        if self.section == "pallavi":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].pallavi)
        if self.section == "anupallavi":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].anupallavi)
        if self.section == "f_caranam":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].f_caranam)
        if self.section == "l_caranam":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].l_caranam)
        if self.section == "text":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].text)
        # Map the song text to indices of the dictionary
        sequence_string_labels = string_to_labels(sequence_raw_string)
        sequence_length = len(sequence_string_labels) - 1
        
        input_string_labels = sequence_string_labels[:-1]       # All except the last character
        output_string_labels = sequence_string_labels[1:]       # All except the first character
                
        # Pad sequence to max text length
        input_string_labels_padded = pad_sequence(input_string_labels, max_length=self.max_text_len)
        output_string_labels_padded = pad_sequence(output_string_labels, max_length=self.max_text_len, pad_label=-100)
        
        return (torch.LongTensor(input_string_labels_padded),
                torch.LongTensor(output_string_labels_padded),
                torch.LongTensor([sequence_length]) )

    
def post_process_sequence_batch(batch_tuple):
    ''' Padding only upto the max size in the given batch'''
    
    input_sequences, output_sequences, lengths = batch_tuple
    
    # Creates a tuple of each element as a tensor
    splitted_input_sequence_batch = input_sequences.split(split_size=1)
    splitted_output_sequence_batch = output_sequences.split(split_size=1)
    splitted_lengths_batch = lengths.split(split_size=1)

    # Creates a list of each element being a tuple of input, output and batch_length
    training_data_tuples = zip(splitted_input_sequence_batch,
                               splitted_output_sequence_batch,
                               splitted_lengths_batch)

    # Sort them according to sequence length
    training_data_tuples_sorted = sorted(training_data_tuples,
                                         key=lambda p: int(p[2]),
                                         reverse=True)

    splitted_input_sequence_batch, splitted_output_sequence_batch, splitted_lengths_batch = zip(*training_data_tuples_sorted)

    # Concatenate each individual tensors into one tensor
    input_sequence_batch_sorted = torch.cat(splitted_input_sequence_batch)
    output_sequence_batch_sorted = torch.cat(splitted_output_sequence_batch)
    lengths_batch_sorted = torch.cat(splitted_lengths_batch)
    
    # Get the sequence padding only upto the max in the given batch
    input_sequence_batch_sorted = input_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    output_sequence_batch_sorted = output_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    input_sequence_batch_transposed = input_sequence_batch_sorted.transpose(0, 1)
    
    # pytorch's api for rnns needs lengths to be list of int
    lengths_batch_sorted_list = list(lengths_batch_sorted)
    lengths_batch_sorted_list = map(lambda x: int(x), lengths_batch_sorted_list)
    
    return input_sequence_batch_transposed, output_sequence_batch_sorted, lengths_batch_sorted_list


class RNN(nn.Module):
    ''' The Recurrent Neural Network Class'''
    
    def __init__(self, input_size, hidden_size, num_classes, n_layers=2):
        ''' Initializing the neural network with the hyper-parameters'''
        
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.n_layers = n_layers
        
	     	# Instead of creating one-hot encoding which would be too big...
		    # we use embedding which creates a fixed size encoding -> input size(vocab size), output size (encoding size) 
        self.encoder = nn.Embedding(input_size, hidden_size)
        
        # Initializing the LSTM layers
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        
        # Creating a fully connected layer to finally give output with the number of classes we want it to have
        self.logits_fc = nn.Linear(hidden_size, num_classes)
    
    
    def forward(self, input_sequences, input_sequences_lengths, hidden=None):
        
        batch_size = input_sequences.shape[1]
        embedded = self.encoder(input_sequences)

		    # Packing efficiently handles batched sequences with variable lengths i.e the unpadded regions
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_sequences_lengths)
        # Get output and hidden to be used for the next hidden
        outputs, hidden = self.lstm(packed, hidden)
        # Unpack the data into sequences with the padding
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # fully connected layer to get the output into required number of classes
        logits = self.logits_fc(outputs)
        logits = logits.transpose(0, 1).contiguous()   # Data should be contiguous in memory for the loss function
        logits_flatten = logits.view(-1, self.num_classes)
        
        return logits_flatten, hidden


# Getting the input from the CSV files for pallavi, anupallavi, caranam and last caranams
telugu_trainset = LyricsGenerationDataset(csv_file_path='telugu_equal.csv', part='text', artists=None)
p_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='pallavi')
ap_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='anupallavi')
fc_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='f_caranam')
lc_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='l_caranam')

telugu_trainset_loader = torch.utils.data.DataLoader(telugu_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
p_trainset_loader = torch.utils.data.DataLoader(p_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
ap_trainset_loader = torch.utils.data.DataLoader(ap_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
fc_trainset_loader = torch.utils.data.DataLoader(fc_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
lc_trainset_loader = torch.utils.data.DataLoader(lc_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size

data_frame = pd.read_csv('div_songs.csv')['no_caranam']
caranam_mean = np.mean(data_frame)
caranam_std = np.std(data_frame)
data_frame = pd.read_csv('div_songs.csv')['no_ap']
ap_mean = np.mean(data_frame)
ap_std = np.std(data_frame)
# p_trainset_loader = torch.utils.data.DataLoader(p_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# ap_trainset_loader = torch.utils.data.DataLoader(ap_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# fc_trainset_loader = torch.utils.data.DataLoader(fc_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# lc_trainset_loader = torch.utils.data.DataLoader(lc_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size

('Part:', 'text')
('max_text_len', 2677)
('Part:', 'pallavi')
('max_text_len', 158)
('Part:', 'anupallavi')
('max_text_len', 177)
('Part:', 'f_caranam')
('max_text_len', 1357)
('Part:', 'l_caranam')
('max_text_len', 397)


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

^C


In [8]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
!ls

Saving telugu_equal.csv to telugu_equal.csv
datalab  div_songs.csv	telugu_equal.csv


In [0]:
lyrics_dataframe = pd.read_csv('div_songs.csv')['no_caranam']
print (np.mean(lyrics_dataframe))

3.026760563380282


In [0]:
# Creating 4 RNNs for pallavi, anupallavi, all but the last caranam and the last caranam
# input_size : size(vocabulary)+1 to account for the padding const
telugu_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
telugu_rnn.cuda()
p_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
# rnn = torch.load('conditional_rnn.pth')
p_rnn.cuda()
ap_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
ap_rnn.cuda()
fc_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
fc_rnn.cuda()
lc_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
lc_rnn.cuda()

learning_rate = 0.001
telugu_optimizer = torch.optim.Adam(telugu_rnn.parameters(), lr=learning_rate)
p_optimizer = torch.optim.Adam(p_rnn.parameters(), lr=learning_rate)
ap_optimizer = torch.optim.Adam(ap_rnn.parameters(), lr=learning_rate)
fc_optimizer = torch.optim.Adam(fc_rnn.parameters(), lr=learning_rate)
lc_optimizer = torch.optim.Adam(lc_rnn.parameters(), lr=learning_rate)

# Loss function
criterion = torch.nn.CrossEntropyLoss().cuda()

def sample_from_rnn(starting_sting="", sample_length=200,sample_end="<EOP>", temperature=1, rnn=p_rnn, only_sample=0):
    ''' Sampling text from the RNN'''
    
    sampled_string = starting_sting
    hidden = None
    
    #Sending initial input as the starting string
    first_input = torch.LongTensor( string_to_labels(starting_sting)).cuda()
    first_input = first_input.unsqueeze(1)   # addds another dimension [1,2,3] -> [[1],[2],[3]]
    current_input = Variable(first_input)

    # Forward propogation
    output, hidden = rnn(current_input, [len(sampled_string)], hidden=hidden)
    
    # Get the last character in the output
    output = output[-1, :].unsqueeze(0)    # unsqueeze adds another dimension [1,2,3] -> [[1,2,3]]

    if rnn == lc_rnn and only_sample == 1:
        while (sampled_string.find("tyAga") == -1):
            sampled_string = starting_sting
            for i in xrange(sample_length):
        #  Softmax function gets values b/w 0 & 1 and the sum of the values in the tensor sum up to 1
        # basically gets the probabilities of each character
                output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
                predicted_label = torch.multinomial(output_dist, 1)          # Multinomial distribution 1 -> number of samples to draw
                sampled_string += all_characters[int(predicted_label[0])]    # Get the next character
                current_input = Variable(predicted_label.unsqueeze(1)) 
                output, hidden = rnn(current_input, [1], hidden=hidden)      # Send the predicted character as input back into the rnn to get the next character
                if (sampled_string[-5:] == sample_end):
                  break
        return sampled_string

    for i in xrange(sample_length):
        #  Softmax function gets values b/w 0 & 1 and the sum of the values in the tensor sum up to 1
        # basically gets the probabilities of each character
        output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
        predicted_label = torch.multinomial(output_dist, 1)          # Multinomial distribution 1 -> number of samples to draw
        sampled_string += all_characters[int(predicted_label[0])]    # Get the next character
        current_input = Variable(predicted_label.unsqueeze(1)) 
        output, hidden = rnn(current_input, [1], hidden=hidden)      # Send the predicted character as input back into the rnn to get the next character
        if (sampled_string[-5:] == sample_end):
            break
    return sampled_string  

In [1]:
!pip install torch==0.4

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x55eaf3ada000 @  0x7f50cbb801c4 0x55ea97f410d8 0x55ea9802ad5d 0x55ea97f5477a 0x55ea97f59462 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f51b3a 0x55ea97f59e1f 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f59462 0x55ea97f59462 0x55ea97f51b3a 0x55ea97f59e1f 0x55ea97f59462 0x55ea97f51b3a 0x55ea97f59e1f 0x55ea97f51b3a 0x55ea97f59e1f 0x55ea97f51b3a 0x55ea97f5982e 0x55ea97f51b3a 0x55ea97f8250f 0x55ea97f7d202


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# %matplotlib notebook

# from matplotlib import pyplot as plt

# # importing a list that can be binded with a figure and update
# # the figure when being appended
# from visualization import VizList

# # Creating figure, axes and binding to lists 
# f, (loss_axis, validation_axis, train_axis) = plt.subplots(3, 1)

# loss_axis.plot([], [])
# validation_axis.plot([], [])
# train_axis.plot([], [])


# loss_list = VizList()
# validation_list = VizList()
# train_list = VizList()

# loss_list.bind_to_axis(loss_axis)
# validation_list.bind_to_axis(validation_axis)
# train_list.bind_to_axis(train_axis)

# loss_axis.set_title('Training Loss')
# validation_axis.set_title('Validation Set Accuracy')
# train_axis.set_title('Training Set Accuracy')


# plt.tight_layout()

In [12]:
''' Telugu Data training'''

clip = 1.0
epochs_number = 1000

for epoch_number in xrange(epochs_number):

    print ("epoch_number", epoch_number)
    for telugu_batch in telugu_trainset_loader:
#         print ("batch.shape", batch[0].shape, batch[1].shape, batch[2].shape)
        
        post_processed_batch_tuple = post_process_sequence_batch(telugu_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        telugu_optimizer.zero_grad()
        telugu_logits, _ = telugu_rnn(input_sequences_batch_var, sequences_lengths)
        telugu_loss = criterion(telugu_logits, output_sequences_batch_var)
        telugu_loss.backward()
        telugu_optimizer.step()

    print(sample_from_rnn(rnn=telugu_rnn, starting_sting="kAga"))
    torch.save({'telugu_rnn':telugu_rnn.state_dict()}, 'telugu_conditional_rnn.pth')
#     torch.save(rnn.state_dict(), 'conditional_rnn.pth')
    
    

('epoch_number', 0)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


kAgalugu sannayugapu vukka raavu aaTalanu unnaayi 
chaanya teevra lO aDigitae daevatalu vivariMchabaDinadi kaMTaegaa bhautika raajadhaananulaa aakulabOyae aastiraachukunna lOpaerani prachuri pani chaesuku
('epoch_number', 1)
kAga janawa guMtAyatAnaku kepada kuppAnu pAXAnu kUxavA aMwa adigAdu ani eVvarukani viplagi iMtadiplena nAkoMdu cUrAlani rAjakerullA xayani nuMcAru 
ku koorae aaMtulO pOlee puTTaaraMTulO chOrae Dabbu baTTi
('epoch_number', 2)


Process Process-27:
Process Process-28:
Process Process-26:
Process Process-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target

KeyboardInterrupt: ignored

In [0]:
files.download('telugu_conditional_rnn.pth')

In [15]:
p_rnn.load_state_dict(telugu_rnn.state_dict())
ap_rnn.load_state_dict(telugu_rnn.state_dict())
fc_rnn.load_state_dict(telugu_rnn.state_dict())
lc_rnn.load_state_dict(telugu_rnn.state_dict())

p_optimizer = torch.optim.Adam(p_rnn.parameters(), lr=learning_rate)
ap_optimizer = torch.optim.Adam(ap_rnn.parameters(), lr=learning_rate)
fc_optimizer = torch.optim.Adam(fc_rnn.parameters(), lr=learning_rate)
lc_optimizer = torch.optim.Adam(lc_rnn.parameters(), lr=learning_rate)

clip = 1.0
epochs_number = 1000

for epoch_number in xrange(epochs_number):

    print ("epoch_number", epoch_number)
    for p_batch in p_trainset_loader:
#         print ("batch.shape", batch[0].shape, batch[1].shape, batch[2].shape)
        
        post_processed_batch_tuple = post_process_sequence_batch(p_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        p_optimizer.zero_grad()
        p_logits, _ = p_rnn(input_sequences_batch_var, sequences_lengths)
        p_loss = criterion(p_logits, output_sequences_batch_var)
        p_loss.backward()
        p_optimizer.step()
        
    for ap_batch in ap_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(ap_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        ap_optimizer.zero_grad()
        ap_logits, _ = ap_rnn(input_sequences_batch_var, sequences_lengths)
        ap_loss = criterion(ap_logits, output_sequences_batch_var)
        ap_loss.backward()
        ap_optimizer.step()
        
    for fc_batch in fc_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(fc_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        fc_optimizer.zero_grad()
        fc_logits, _ = fc_rnn(input_sequences_batch_var, sequences_lengths)
        fc_loss = criterion(fc_logits, output_sequences_batch_var)
        fc_loss.backward()
        fc_optimizer.step()
        
    for lc_batch in lc_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(lc_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        lc_optimizer.zero_grad()
        lc_logits, _ = lc_rnn(input_sequences_batch_var, sequences_lengths)
        lc_loss = criterion(lc_logits, output_sequences_batch_var)
        lc_loss.backward()
        lc_optimizer.step()

        #torch.nn.utils.clip_grad_norm(rnn.parameters(), clip)

    # Getting the number of caranams
    no_caranam = max(1, int(np.ceil(np.random.normal(caranam_mean, caranam_std))))
    
    # Sampling pallavi
    print(sample_from_rnn(rnn=p_rnn, starting_sting="pallavi\n"))
    
    # Sampling anupallavi
    if np.random.normal(ap_mean, ap_std) > ap_mean:
      print(sample_from_rnn(rnn=ap_rnn, starting_sting="anupallavi\n"))
    
    # Sampling first_caraNam
    for i in range(no_caranam-1):
      print(sample_from_rnn(rnn=fc_rnn, starting_sting="caraNam " + str(i+1) + "\n"))
    # Sampling last caraNam
    if (epoch_number >= 800):
      if no_caranam == 1:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>", only_sample=1))
      else:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>", only_sample=1))
    else:
      if no_caranam == 1:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>"))
      else:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>"))
      
    torch.save({'prnn':p_rnn.state_dict(), 'aprnn':ap_rnn.state_dict(), 'fcrnn':fc_rnn.state_dict(), 'lcrnn':lc_rnn.state_dict()}, 'conditional_rnn.pth')
#     torch.save(rnn.state_dict(), 'conditional_rnn.pth')
    
    

('epoch_number', 0)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pallavi
shAvamu galasE juiccAnu jElurAdimmu TAthacdu tithamulamEmmma pApanda bhAraghavarA PEOP<
veva nAvaruPE tanamu jErati kaLlanu nI sanhanatuntarA venutE rAma sagamulaitha pAgaLtuku gali I nIku pAlanE namm
caraNam
tyAgarAja samatyumbuDitida vA
tinta parakka bhakta caba Odara ganuNa bhaktamu epAmulate nam
nAma AsatAni
Acumati KAddOy>ni panmalaikoddi baNcu nergaganu brOdamulaki unna rAma pramasAga dharma prashard
('epoch_number', 1)
pallavi
aruPiti puta shA jatA ninnulu prANa jilaja unkuDE velarP< unnAmE sAmE mAm svAbhivE shrI rAma rAma nidarAma GEOP>
tAmulu mani snrashvara racitramunu E <EOP>
caraNam 1
rAn>la On=tina caDugA kanPOrimi mAlAyula ganuDEni prakAma nAmatula papEsupava tavyamulu januni HeuPO 
sokkula pAmanaina pAluuutanugaitAnan karEkhAla parAvayaSara dhAshraSAga telaNAlu mariyagnaTT
talug
caraNam 2
sistAru va samEkaLincu vima magamulalO 
jUu paSEjanu vibhE vara
nu rammina brO gAraga vanduti nAvanadamulatO cAramayya ilalO ikOtivanulu nijAra balukonda ga-gamukE nAjanutO okAn

caraNam 5
shrIpati jEsIya phrinci sakala baTTu vELa mUrjanula kalugu shUra hiSayuNTE vittajAlana pEruSaka vaitramandu gAka vEDapaTa gaNa vinAthAnandhi sEkuSata <EOP>
caraNam 6
mArika shAstra vidukulu vari tyAgarAja nutuni daya vEbhASa tyAgarAjarekula nenaru okA dArana migujampaga lEdani 
tyAgarAjanuta <EOS>
('epoch_number', 11)
pallavi
innALLu nOrvinnucu varamuna nEnu <EOP>
anupallavi
sarapatradina nIradhi gAni shIla danda janaka <EOP>
caraNam
prahlAdA nagajArcita shbItakaNAgatAkSTula nru-tAra vAdA telisi nIravakai pApAya 
nirguNa shrI telipaa tAra munikice- bhAvEmi tyAgarAju kApAdi telisi <EOS>
('epoch_number', 12)
pallavi
haryanu bbOdusuru neratvamunu <EOP>
anupallavi
akalamu konnALLu noka nAtO <EOP>
caraNam
bhUsinchaganE tyAgarAju tala ceppukunu <EOS>
('epoch_number', 13)
pallavi
Emani teliyinadendu muryudu lADEda vEDUrA <yAla gAnijaladAshi O <EOP>
anupallavi
kAmakOTi <EOP>
caraNam 1
kuvalana lOyu
vadilella dorakaru nI marma bhIkanAtha <EOP>
caraNam 2
mukhamulaku jUcE <EOP>
caraNam 

caraNam 1
santa vArE nAdu sEyaka bhrnta-mAtO kAjukti <EOP>
caraNam 2
tAgarAja vinuta vara bhaktulu jUci manasu aru jampacElam oka parAkEmi jEsi
tirigirOlulu vara <EOS>
('epoch_number', 21)
pallavi
ninnu vinA narasimha nanu prANanAtha <EOP>
caraNam 1
taraNa varAvara shu-bhrta parula 
varaina vaiviyuNDagnu jUci hrdcelanu baTTulO

ninnu kOva nElamuN munilaagadu bhuvini vinucu 
nI karmamaina sulabham mI bhEmukha vEltuni <EOP>
caraNam 2
krta rAma dayAdulace muddu lADa vESina viDudu mutyadhApa shuka sAmarthita janula nE jAlanaidi sumana <EOP>
caraNam 3
rasiyu balikina meDali ghanamulatO pEdA <EOP>
caraNam 4
jigAna karamniya diSa anaga krpa birAna tApava sEyakarA
nataSAramA nI jAnakSAtmagurA rAmacEta 
nijamAga pAvana tAmasa mAna dhara rAjadhara rAmabOcuna Or janadAdhipa sunta padAlam
sEya mAliyani taraNiy
('epoch_number', 22)
pallavi
teliyalEru rAma nepai manasuku dIruga tArakAdaNa vaipura <EOP>
anupallavi
kantuni madiyaina maNnudikhi yuncani <EOP>
caraNam 1
santasillintu rAma anusari tALa gA

...

caraNam 2
rAvaNAduDu E pApa metayaina jAlamu lADukoni 
brOvu patita pAvani dharma samvardhani <EOP>
caraNam 3
rangapai daya sUryakula pogaDi tirigi dikkulyE gati gAni <EOP>
caraNam 4
shara cApamulu velaya surucira vAdyamulu mrOya surusellaru 
vinuti sEyavara tyAgarAja sannutuni maravakE manasA <EOS>
('epoch_number', 705)
pallavi
rAma rAma rAma rAma aninanta rAjapu jUpElarA O rAma <EOP>
anupallavi
sAkEtAdhipa sarasa gunApramEya sarasijAsana 
sanandana vanditAnghra yuga pada nirjita muni shApa <EOP>
caraNam
vEda varNa nIyamau nAmamuto vidhi rudrulaku mElmiyagu rUpamutO
mOda sadanamagu paTu caritamutO munirAja vEsiyau tyAgarAjanuta ni <EOS>
('epoch_number', 706)
pallavi
pAhi paramAtma satatam mAm <EOP>
anupallavi
krpaku pAtruDanani kIrtincinArA <EOP>
caraNam 1
ramanAmarAdhipa nata kara kundarada balka vadda 
lAdi shrI rAma pAlanaina bhaktula rOsi karmamu bAsi ninu karuNa vArAshi
ani pUja jEsi dussangati jEsi mEnu gAsi jendaka shrIsha devAdhIsha ninu
kAshish
caraNam 2
rArA sAhamukha dhAra 

caraNam 2
rAmamu nitya nirupEmOsamau prahlAduDadu mana vinutinci nI dArina vADu <EOP>
caraNam 3
rangapai virahitulaina narulu nAdupai mari yasUyala baracina 
bAdhalu taramugAka nI caraNa yugamulanu sharaNoNDina nann- <EOP>
caraNam 4
pApamu nI shala nAradamunu nenu nI shubha caritamunu gani nI parimalamunu gani 
panulanu talli taNDri lEni bhaktiyanE krti
mukhulu sancElamu sukhamAnjanEyu bramha kevaru vEra karagu cinta cararEvale n
('epoch_number', 718)
pallavi
nI dayacE rAma nityAnanduDaiti <EOP>
anupallavi
cuTlara gaDiya dOvaku nA madilOni jAli delipi vibhunitO delpuvArin- <EOP>
caraNam 1
rangadhara vinAyakanu janakaja nayana ghana nIkaita rAma
bandhu sarvasvale idi bhuvini sannutaga karagi karagi nilvAga karamu baTTi kaugalincE varaduDappuDu <EOP>
caraNam 2
rAma mannanatO Emani karagaTa jAlEnu nunca pAdavaTAlO <EOP>
caraNam 3
rangapai daya sukhamu sundarAsaDa janaka <EOP>
caraNam 4
rangEshuni AdikuNTE <EOP>
caraNam 5
krta sangIta lOla <EOP>
caraNam 6
rAmacandra nI nAmamu dEva cElamu i

caraNam 4
ranjilalO buTTiti pAhi rAma anucu gaTTi paTTU baTTiti <EOP>
caraNam 5
rangEshuni sharaNAgata vatsalAgama sam <EOP>
caraNam 6
kanilAsa natanamaina sogasunu mari mari ganalEka nenaruna
tyAgarAjanutuni pogaDaka kanarAnidi kanucu vinarAnidi vinucu <EOS>
('epoch_number', 726)
pallavi
mEnu jUci mOsa bOkavE manasA lOnijADa lIlAgu kAdA <EOP>
anupallavi
dEvAdi dEva bhUdEva varapakSa rAjIvAkSa sAdhu jana jIvana sanAtana <EOP>
caraNam
kamanIya jaya dhana dhAnya rUpAm kanakAdi dhairya lakSmI svarUpAm
kamalAm mahA santAna rUpAm kamalAlayAm tyAgarAja mOkSa rUpAm <EOS>
('epoch_number', 727)
pallavi
tuLasi bilva mallikAdi jalaja suma pUjala gaikonavE <EOP>
caraNam 1
ravillAraka vairAciyununa nanukoNTi balukula 
mI anacina nidhi <EOP>
caraNam 2
purANa cittamu sudAnava shrI tyAgarAja sahituDai damadhara dIna <EOS>
('epoch_number', 728)
pallavi
nrpAlavAla kalAdhara shEkhara krtAbhi vandana shrI rAma <EOP>
anupallavi
enduku capalamu vinavE nA manavini mundaTivale bhaktula
pOSincuTak- <EOP>
caraN

caraNam 5
kanulAra shESaTa sadguNa shIla narahara 
sankita vara bhOgishayana nIvE gatiyani baTTi rAgi
yuramuna konnALLu tyAgarAja nutuDaina shrI rAmuni tatvamu telipakanE konnALLu <EOS>
('epoch_number', 736)
pallavi
raghunandana raghunandana raghunandana raghunandana rAjE panjana jEsinadi nanTE nI <EOP>
caraNam 1
ramanAmarAdhipa rAm <EOP>
caraNam 2
rAvaNAduna vairi madi galigina tAvuna poralEDi jUDu <EOP>
caraNam 3
pancanadIsa nata janaabhra sannuta munijana sura pAlaka shankara nara san-pAnanda mrdu bhASaNa tyAgarAja vinuta hrdAlamula <EOS>
('epoch_number', 737)
pallavi
ennALLu tirigEdi ennALLu <EOP>
caraNam 1
rivarNasthita manOharA sundara rAma <EOP>
caraNam 2
ranjilla dindukA <EOP>
caraNam 3
krovvulaku mari ghOmula
merugu bangAru cEla miravonda mariyu nUpuramunu ghallanaga bhaktula gani
karuNAmrtamu callaga yOgula daharamulu jhallanaga vEncEsina <EOP>
caraNam 4
stramuguja-nu bhramasi nEnu jUcucu vadda nilici vAramu pUjincE <EOP>
caraNam 5
rangEshuni shruti manasuna ninu kula tilakam

KeyboardInterrupt: ignored

In [19]:
    ''' Sampling '''
    for i in range(1000):
        print ("example", i)
        # Getting the number of caranams
        no_caranam = max(1, int(np.ceil(np.random.normal(caranam_mean, caranam_std))))
    
        # Sampling pallavi
        print(sample_from_rnn(rnn=p_rnn, starting_sting="pallavi\n"))
    
        # Sampling anupallavi
        if np.random.normal(ap_mean, ap_std) > ap_mean:
            print(sample_from_rnn(rnn=ap_rnn, starting_sting="anupallavi\n"))
    
        # Sampling first_caraNam
        for i in range(no_caranam-1):
            print(sample_from_rnn(rnn=fc_rnn, starting_sting="caraNam " + str(i+1) + "\n"))
        # Sampling last caraNam
        if (epoch_number >= 800):
            if no_caranam == 1:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>", only_sample=1))
            else:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>", only_sample=1))
        else:
            if no_caranam == 1:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>"))
            else:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>"))
        print("\n\n")

('example', 0)
pallavi
cakkani rAja mArgamu lunDaga sandula dUranela O manasA <EOP>
anupallavi
lAli yugavE nanugAli diNDi pAnpumani <EOP>


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


caraNam 1
ripaddu lanka paripUrNa nAdaya <EOP>
caraNam 2
rAma pOSinci nA manasu kAla nanEtharADu <EOP>
caraNam 3
pAvana samhAram rAghavam udAram tyAgarAja shayana vinarA jagatpati 
rAja padam bhAgadEya rAja bhava <EOS>



('example', 1)
pallavi
Ela teliyalerO pUrva karmamElA-guNa jEsirO rAmayya <EOP>
anupallavi
ajuDaina harihayuDaina nA bhaktiyu <EOP>
caraNam 1
rivaddhi varanucu brOturaga vihArA jalajamaraga yOcana 
sharaNa <EOP>
caraNam 2
rArA shrI raghupa sukhamutO skAtuni vanani taNDrulu anna bhakti 
gajA pAlanamu sukhamu sairincukonna saundaramba lEdu <EOP>
caraNam 3
rangapai nuta vibhUSaNuniki mangaLam <EOP>
caraNam 4
rangEshuDu valacungani gala vAsanuDu cAla goTTi cuTTivuluku cAla gUDa valaciti gAni 
kAnciti nIvE gAni pAdamula celagu suranga mrOsanu kAvale samayamuna rAvE <EOP>
caraNam 5
kOTa losagi konnALAsura jAma pUrituDagu tyAgarAja hrdaya nivAsa sannuta <EOS>



('example', 2)
pallavi
shrI rAma pAdamA nI krpa jAlunE cittAniki rAvE <EOP>
caraNam 1
ranganAyaka hari paripUrNa s

anupallavi
vATAshanAri vara vAhana vArijAsanAdi vanditapada <EOP>
caraNam 1
rivarNayunu jaripi vara vArAshi janitE kanaka bhUSaNa lasitE kalashaja
gItamuditE kAkustharAja sahitE <EOP>
caraNam 2
rArA surulu nOrula mIda unDa viDa mIdu <EOP>
caraNam 3
rangapai daya sukhamu saligE <EOP>
caraNam 4
rangEshuvarA shrI rAma <EOP>
caraNam 5
krovvulaku parama vaddanE lOkuncandruni mangaLam shubha mangaLam <EOP>
caraNam 6
ganga haraNa bhUSa parAtpara tyAgarAjArcita <EOS>



('example', 12)
pallavi
gurulEka eTuvaNTi guNiki deliyaga bOdu <EOP>
caraNam 1
ramanAmari pUla pAla virahita dAsharathE <EOP>
caraNam 2
rArA surulu bharva santatamu <EOP>
caraNam 3
krtakuDu balamu kAsanga manamu <EOP>
caraNam 4
rangEshuDu sagha gAtruni ApadEshincina <EOP>
caraNam 5
kAmala madamella jarigenu tyAgarAja nutuni <EOS>



('example', 13)
pallavi
paripAlaya dAsharathE rAma mAm pAhi paripAlaya dAsharathE <EOP>
anupallavi
ElAgu ninnADukonna nEramenca pani lEdu nApaini <EOP>
caraNam 1
ripaluku vacanamulu viDucuna <EOP>
c

caraNam 4
rangEshuDu sadgangA janakuDu sangIta sAmpradAyakuDu <EOP>
caraNam 5
krtakuDu tAmasamu shrI lOla nE neppuna narulu gAna <EOP>
caraNam 6
kali puruSuDu nATakamunu gaTTa dalacinADu shrI tyAgarAjanuta
khala matamulane yAgamulaku mEkaluga narula jEsi nI dayajUDa <EOS>



('example', 23)
pallavi
Emi jEsitE nEmi shrI rAma svAmi karuNa lEni vArilalO <EOP>
anupallavi
shrImantulu nI sOdrulu sEyurIti pAda sEva kOritini <EOP>
caraNam 1
rikalini pOrcina dala gani <EOP>
caraNam 2
rAvaNAduDu E pApa vinutunakai rAdayya rAya magu pApamE lEka uNDani bhaktula vatiyADa
vara dhrta virata kAryamu sallipu karagi bhuvilO rAjasammuna vIrula
kusutu mUpu satsalam
kAmAdula celimimi 
shrI rA
caraNam 3
krnagnAya dayA parAtpara nannu tana durAsala dhana dAsa lEdA <EOP>
caraNam 4
sariga jAlamulanu jUcuna dennaTikO 
kOpamuna bhrgu sutu cApa balamandu konna bahunu jUcuna dennaTikO <EOP>
caraNam 5
rangEra balka bAdhalO <EOP>
caraNam 6
rangEsava rAjAntuniki santata panulanu jUci 
endubOka rAkEndu mukhuni tanayan

...

anupallavi
uNDi Emi urvi bhAramugA kOdaNDapANini kanulaniNDa cUDa vAr- <EOP>
caraNam 1
ripalukula nagra yA harE sAramu bAla 
gamana nAma smaraNamulu mari janulati bhEdamula gaDipE dAri <EOP>
caraNam 2
rAma shApamutO shArkka sanamula vADaDu jIvan muktiDai tirugu mudamunu jUpaDu <EOP>
caraNam 3
padAsala caraNa tApasa mAnasa nIraja lOcana mAku vAji kari dhanamulu varamu gAdurA 
jAji sumadharaNa bhuja manOhara shrI tyAgarAjanuta bahu ravi nA tappu lencaka <EOS>



('example', 945)
pallavi
AtadE dhanyudurA O manasA <EOP>
caraNam 1
ramanya ani bAluDa gAni <EOP>
caraNam 2
rAma mannanaku mati bAdhala nella dalaci nanu <EOP>
caraNam 3
rangapamaina  sanandana nutamAnanda jaladhi mahi brnda su-bhUSaNa vandita caritam <EOP>
caraNam 4
rangEshuDu sadgangA janakuDu sangIta sAmpradAyakuDu <EOP>
caraNam 5
rangEshuni shruti mantra bhAgyamaina sundaramaina nannaga jEyu suNDu bhAmini sadA mariyAdA <EOP>
caraNam 6
rangEsa viSva shrI dAsa tIraNa vyasandhuDu nI shaktiki dhruvuDu sAkSi gAdA rAma nanu <EOP>
ca

caraNam 10
mukha dayarAdanula pUjinca nEnu dInAnadanu dEvadEva
jAjikAyulu elakulu jApatri vakka lAkulu rAdu tyAgarAja nutuni dayalEni vArilalO <EOS>



('example', 956)
pallavi
daya jUcuTa kidi vELarA dAsharath <EOP>
caraNam 1
rangadhara vinuta <EOP>
caraNam 2
rArA shrI raghupa sukhamutavalaya sarulalO rAra manEsamella durAsala jAla <EOP>
caraNam 3
rangapai daya rUpa shrI rAma  <EOP>
caraNam 4
sariga jUci sirulanu O rAma <EOP>
caraNam 5
kalashAmbudhilO dayatO namarulakai adigAka gOpi
kalakai koNDa lella lEdA shrIkAndu lAganayya 
venvayamEla rahita manasuna buTTina nEnu jAlini shrI tyAgarAja varaduDu tAn- <EOS>



('example', 957)
pallavi
enduku peddala vale buddhi iyavu endu bOdu nayyA rAmayyA <EOP>
anupallavi
toli karma managa jUtamu rArA tOyajAri rOhini gUDina rIti <EOP>
caraNam
ennarAni janana maraNamu lEkuNDa manasAra tyAgarAjanutuni paini pUla <EOS>



('example', 958)
pallavi
nAdupai balikEru narulu vEdasannuta bhavamu vEru jEsiti nanucu <EOP>
anupallavi
aja gaja rakSaka gaja car

caraNam 2
rAmamO tOcadAra sAreku nAyenu nIku proddu pogaDi pogaDikkina pushamula jEritivA pAlu kushIla mA ramaNa <EOP>
caraNam 3
rangapai daLita apuDE mATala digahOda mada bhavasAgara brahAdya
bhEdAravamu <EOP>
caraNam 4
sariga jUci nA sara sarOja bhuvya tagunAya bhayamanditO evari shrI dAsa rUpamu dA
vastiru madini nIkEla nindini muktAtbarandamu gAdu sudhAkarma ninu vEra <EOP>
caraNam 5
ramanammi buddhu lOka karmamu jEsina stramaina gAni <EOP>
caraNam 6
rArA bhAvaNri hrdaya vargAdula subhASa 
sAmha shubhakara vESa nirdOSa <EOP>
caraNam 7
tAramani tatvamu delisi bhInOlula nAlabhEdamanE tyAgarAjanuta <EOS>



('example', 970)
pallavi
rAma bANa trANa shaurya mEmani pogaDudurA O manasA <EOP>
caraNam 1
ramanAmara suguNa dharalO galugu napurinunucu 
satyamunu garjana gAmi jUcuna dennaTikO <EOP>
caraNam 2
rArA shrI rAma nA manasuna nanu driTa nannu brOva vEga rAdA <EOP>
caraNam 3
krtakuDu balamaina ninnu kuluva vayAlu sutA dashamulanu nE vidhuni bAdhalu dharma sampadamu veravaka 
bOnEradu en

caraNam 4
sariga jUci nA sara sandurA samudra <EOP>
caraNam 5
kannulanEDi yampa kolacEta gucci canulanEDi girula shiramuninci 
panulu cEturaTa tyAgarAja nutuni bAga nIvu bhajana jEsukommi strIla <EOS>



('example', 980)
pallavi
enta vEDukondu O rAghava pantamElarA O rAghava <EOP>
anupallavi
shrI nAthu koluvamare jelula cekkillu noTTuccunu
mAnaka mOvAnucu candrAnanu hrdayamuna nunca <EOP>
caraNam 1
ramanAmara suguNa viSaya sahAsadAshula gAni <EOP>
caraNam 2
rArA surulu bharAja sumamulu sarulala valvaga jEyucunu okatE vEDkaga <EOP>
caraNam 3
rangapai daya sokka kAlavAsuni sItA <EOP>
caraNam 4
rangEsarva kOTulalO teliyakanu bhava sukhamula kAshincina gAni 
dhruvamaina phalamosagu nI shaktiki dhruvuDu sAkSi gAdA rAma nanu <EOP>
caraNam 5
rangEshuni bahu vakkalga ghana mada selavai vindamuna koraku dayacEsi taruNula vashamA <EOP>
caraNam 6
rangEsava rAjasamauni varamaDuga vanita rUpuDAyE Ashinchi
durvAsulu annamaDuga apuDE mandamAyE  O rAma nin <EOP>
caraNam 7
karanIyuniki vinu mannimpa nI

caraNam 2
shukati nEva rangamuna vara nAradAmshuDai avatarinci rasamuga gIta 
shAstra marmamulanu ranjillaga manujulaku bOdhinci vasudEva tanayuDagu shrInivAsa vajIruni 
okka krpaka pAtruDai asamunu nirantaramu



('example', 990)
pallavi
lAli yugavayyA nA pAli daivamA rAmA <EOP>
caraNam
nE sEyu duSkarma dEvata nI sannidhini nilva sAgenO 
nI sEva nilvadElarA nIrajAkSa tyAgara sannuta <EOS>



('example', 991)
pallavi
vinatA suta rArA nA vinuti gaikonarA <EOP>
anupallavi
aDiyAsalacE dagili nE nArtipaDinA padambulanu <EOP>
caraNam 1
ramanAmarAdhipa samhAra nagaja purakaraga janukula celimi oka padavA <EOP>
caraNam 2
satyamu tAraka nAma sucaritra gaNapati hrdayamu niNDi 
prEma buTTa sEyaga lEdA niSkAma tyAgarAju sEyu nAma <EOS>



('example', 992)
pallavi
brndAvana lOla gOvindA-ravinda nayana <EOP>
caraNam
I jagatini tyAgarAjuni brOcuTaki jAla mika tALanu nI vELanu <EOS>



('example', 993)
pallavi
dorakunA iTuvanTi sEva <EOP>
anupallavi
kanna kannavArini vEDu konnAnu phalamu lEdani nE <E

In [0]:
# !ls
# from google.colab import files
# files.download('conditional_rnn.pth')
a = torch.tensor([1,2,3])
b = a.split(split_size=1)
# output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
# predicted_label = torch.multinomial(output_dist, 1)


[(1, 4, 7), (2, 5, 8), (3, 6, 9)]